In [4]:
from Bio.PDB import *
import urllib.request
import numpy as np
import pandas as pd
from math import sqrt
import time
import configparser
import os
import pymysql

In [5]:
config = configparser.ConfigParser()
config.read("/home/yuan/Documents/config.ini")
config.sections()

['office_ssh', 'office_mysql']

In [6]:
modification_site = pd.read_csv("/home/yuan/Documents/pdb_chain_ksites.csv")

In [7]:
modification_site[0:5]

,PDB,CHAIN,SP_PRIMARY,RES_BEG,RES_END,PDB_BEG,PDB_END,SP_BEG,SP_END,Ksites
0,3g8c,A,P24182,1,444,1,444,1,444,27
1,3g8c,B,P24182,1,444,1,444,1,444,27
2,3g8d,A,P24182,1,444,1,444,1,444,27
3,3g8d,B,P24182,1,444,1,444,1,444,27
4,3jzf,A,P24182,22,470,1,449,1,449,27


In [8]:
modification_site_dic = {}
for i in range(len(modification_site)):
    if modification_site.loc[i,"PDB"] not in modification_site_dic:
        modification_site_dic[modification_site.loc[i,"PDB"]] = {modification_site.loc[i,"CHAIN"]:[modification_site.loc[i,"Ksites"]]}
    elif modification_site.loc[i,"CHAIN"] not in modification_site_dic[modification_site.loc[i,"PDB"]]:
        modification_site_dic[modification_site.loc[i,"PDB"]][modification_site.loc[i,"CHAIN"]] = [modification_site.loc[i,"Ksites"]]
    else:
        modification_site_dic[modification_site.loc[i,"PDB"]][modification_site.loc[i,"CHAIN"]].append(modification_site.loc[i,"Ksites"])
for protein in modification_site_dic:
    for chain in modification_site_dic[protein]:
        modification_site_dic[protein][chain] = list(modification_site_dic[protein][chain])
        

In [9]:
unique_list = modification_site[["PDB","CHAIN"]].drop_duplicates()

In [10]:
unique_list = unique_list.reset_index(drop=True)

In [11]:
backbone = ["N","CA","C","O"]
aminoAcidCodes = ["ALA","ARG","ASN","ASP","CYS","GLN","GLY","GLU","HIS","ILE","LEU","LYS",
                 "MET","PHE","PRO","PYL","SER","SEC","THR","TRP","TYR","VAL"]

In [12]:
def n_nearest_neighbour_by_seq(PDB_ID,Chain,res_ID,n):
    if n >= res_ID:
        return []
    conn = pymysql.connect(host=config["office_mysql"]["sql_host"],user=config["office_mysql"]["sql_username"],
                       passwd=config["office_mysql"]["sql_password"],
                       db=config["office_mysql"]["sql_main_database"],port=3306)
    try:
        with conn as cursor: #auto commit; no close() called
            with cursor: # close() called here
                sql_select = "Select da.* FROM pdbdb.`Distance_angle_PTM` da "
                sql_where = "WHERE da.pdbID = \"{}\" and da.chain = \"{}\" and da.ID_1= \"{}\";".format(PDB_ID,Chain,res_ID)  
                sql = sql_select+sql_where
                data = pd.read_sql_query(sql, conn)
        head_part = data.groupby(["ID"]).first().iloc[(res_ID-n-1):(res_ID-1),]
        tail_part = data.groupby(["ID"]).first().iloc[res_ID:(res_ID+n),]
        return(pd.concat([head_part,tail_part])) #from 2 to exclude self vs self

    except Exception as e: # catch exceptions
        print("~~~~~~~~~~~~~~")
        print(e)
    finally:
        if conn:
            conn.close()

In [13]:
n_nearest_neighbour_by_seq("3g8c","A",15,5)

,pdbID,chain,ID_1,Res_1,ID_2,Res_2,Distance,Angle
ID,,,,,,,,
6226,3g8c,A,15,LEU,10,ARG,11.217045,76.474551
6227,3g8c,A,15,LEU,11,GLY,7.209092,86.154560
6228,3g8c,A,15,LEU,12,GLU,7.610891,88.116599
6229,3g8c,A,15,LEU,13,ILE,9.033376,47.692547
6230,3g8c,A,15,LEU,14,ALA,6.644277,43.795225
6232,3g8c,A,15,LEU,16,ARG,8.508435,75.514795
6233,3g8c,A,15,LEU,17,ILE,9.106243,19.250362
6234,3g8c,A,15,LEU,18,LEU,5.940790,84.540007
6235,3g8c,A,15,LEU,19,ARG,4.270894,130.326129


In [14]:
def One_residue_retrieval(residue_1,PDB_ID,Chain):
    data, data_1, data_2 = None, None, None
    conn = pymysql.connect(host=config["office_mysql"]["sql_host"],user=config["office_mysql"]["sql_username"],
                           passwd=config["office_mysql"]["sql_password"],
                           db=config["office_mysql"]["sql_main_database"],port=3306)
    try:
        with conn as cursor: #auto commit; no close() called
            with cursor: # close() called here
                sql_select = "Select da.* FROM pdbdb.`Distance_angle_PTM` da "
                sql_where = "WHERE da.pdbID = \"{}\" and da.chain=\"{}\" and da.ID_1 = {};".format(PDB_ID,Chain,residue_1)    
                sql = sql_select+sql_where
                data_1 = pd.read_sql_query(sql, conn)
                data = data_1
        
    except Exception as e: # catch exceptions
        print("~~~~~~~~~~~~~~")
        print(e)
    finally:
        if conn:
            conn.close()
        return(data)

In [15]:
result_step1 = One_residue_retrieval(144,"3g8c","A")
result_step2 = result_step1.groupby(["ID"]).first() #duplicate discard
result_step3 = result_step2[~result_step2.ID_2.isin([144])] # activate discard
result_step4 = result_step3[result_step3["Res_1"]==result_step3["Res_2"]].sample(3,random_state =1) #sample nonactive site
result_step4.sample(2,random_state =1).iterrows()

<generator object DataFrame.iterrows at 0x7f02a9b46e60>

In [16]:
result_step4.sample(2,random_state =1)

,pdbID,chain,ID_1,Res_1,ID_2,Res_2,Distance,Angle
ID,,,,,,,,
63907,3g8c,A,144,LYS,415,LYS,53.256177,44.854980
63532,3g8c,A,144,LYS,40,LYS,58.867774,67.357118


In [17]:
pos_sample_and_neig = pd.DataFrame()
for key, value in modification_site_dic.items():
    for chain, sites in value.items():
        for site in sites:
            pos_sample_and_neig = pd.concat([pos_sample_and_neig,n_nearest_neighbour(key,chain,site,1)])

NameError: name 'n_nearest_neighbour' is not defined

#### Now, positive sites are ready. Negative sites are going to be picked. 
Right now, one pos - one neg

In [16]:
neg_sample_and_neig = pd.DataFrame()
for i in range(len(pos_sample_and_neig)):
    result_step1 = One_residue_retrieval(pos_sample_and_neig.iloc[i,2],pos_sample_and_neig.iloc[i,0],pos_sample_and_neig.iloc[i,1])
    
    result_step2 = result_step1.groupby(["ID"]).first() #duplicate discard

    result_step3 = result_step2[~result_step2.ID_2.isin(modification_site_dic[pos_sample_and_neig.iloc[i,0]][pos_sample_and_neig.iloc[i,1]])] # activate discard
    #print(result_step3)
    result_step4 = result_step3[result_step3["Res_2"]=="LYS"]
    #print(result_step4)
    #print(result_step4.sample(1,random_state = 1))
    neg_sample_and_neig = pd.concat([neg_sample_and_neig,result_step4.sample(1)])

##### Res_2 in neg_sample_and_neig is the negative sample

In [18]:
pos_sample_and_neig_pruned = pos_sample_and_neig.iloc[:,[0,1,2,3]].copy(deep=True)
pos_sample_and_neig_pruned["type"] = "Pos"
pos_sample_and_neig_pruned.columns = ['pdbID', 'chain', 'ID', 'Res', 'type']

In [19]:
neg_sample_and_neig_pruned = neg_sample_and_neig.iloc[:,[0,1,4,5]].copy(deep=True)
neg_sample_and_neig_pruned["type"] = "Neg"
neg_sample_and_neig_pruned.columns = ['pdbID', 'chain', 'ID', 'Res', 'type']

In [20]:
pos_neg_sample = pd.concat([pos_sample_and_neig_pruned,neg_sample_and_neig_pruned])

In [21]:
neighbour_information_list = []
for i in range(0,len(pos_neg_sample)):
    neigh = n_nearest_neighbour(pos_neg_sample.iloc[i,0],pos_neg_sample.iloc[i,1],pos_neg_sample.iloc[i,2],5).iloc[:,5:8].values.flatten()
    neighbour_information_list.append(np.insert(neigh,0,pos_neg_sample.iloc[i,3]))

In [18]:
import pandas as pd
import os
import numpy as np
from datetime import datetime

from sklearn.metrics import classification_report, roc_auc_score, roc_curve, make_scorer, confusion_matrix, \
    recall_score, precision_score, f1_score
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit, GroupKFold, cross_val_score
from sklearn.metrics import auc, precision_recall_curve,accuracy_score
from sklearn.utils import shuffle
from sklearn import tree, svm, naive_bayes, neighbors
from sklearn.ensemble import BaggingClassifier, AdaBoostClassifier, RandomForestClassifier, GradientBoostingClassifier
from sklearn.preprocessing import LabelBinarizer, OneHotEncoder,LabelEncoder
from scipy import interp
import matplotlib.pyplot as plt

In [19]:
clfs = {'decision_tree': tree.DecisionTreeClassifier(),
        'naive_gaussian': naive_bayes.GaussianNB(),
        'naive_mul': naive_bayes.MultinomialNB(),
        'K_neighbor': neighbors.KNeighborsClassifier(),
        'bagging_knn': BaggingClassifier(neighbors.KNeighborsClassifier(), max_samples=0.5, max_features=0.5),
        'bagging_tree': BaggingClassifier(tree.DecisionTreeClassifier(), max_samples=0.5, max_features=0.5),
        'random_forest': RandomForestClassifier(n_estimators=50),
        'adaboost': AdaBoostClassifier(n_estimators=50),
        'gradient_boost': GradientBoostingClassifier(n_estimators=50, learning_rate=1.0, max_depth=1, random_state=0),
        
        }
#'svm': svm.SVC(probability=True)

In [20]:


aminoAcidCodes = ["ALA","ARG","ASN","ASP","CYS","GLN","GLY","GLU","HIS","ILE","LEU","LYS",
                 "MET","PHE","PRO","PYL","SER","SEC","THR","TRP","TYR","VAL"]
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(aminoAcidCodes)
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit(integer_encoded)
print(onehot_encoded)



OneHotEncoder(categorical_features=None, categories=None,
       dtype=<class 'numpy.float64'>, handle_unknown='error',
       n_values=None, sparse=False)


/home/yuan/anaconda3/envs/pdbPy3/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [21]:
amino_acid_property_by_res = {"ALA":1,"ILE":1,"LEU":1,"MET":1,"VAL":1,
                      "PHE":2, "TRP":2,"TYR":2,
                       "ASN":3,"CYS":3,"GLN":3,"SER":3,"THR":3,
                      "ASP":4,"GLU":4,"ARG":5,"HIS":5,"LYS":5,"GLY":6,"PRO":6}
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform([1,2,3,4,5,6])
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit(integer_encoded)
print(onehot_encoded)



OneHotEncoder(categorical_features=None, categories=None,
       dtype=<class 'numpy.float64'>, handle_unknown='error',
       n_values=None, sparse=False)


/home/yuan/anaconda3/envs/pdbPy3/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [ ]:
train_dataset = []
for each_res in neighbour_information_list:
    one_res_data = []
    for element in each_res:
        if isinstance(element,str):
            one_res_data.extend(onehot_encoded.transform([[amino_acid_property_by_res[element]-1]])[0].tolist())
        else:
            one_res_data.append(element)
    train_dataset.append(one_res_data)
wrong_sample = []
for i in range(0,len(train_dataset)):
    if(len(train_dataset[i])<30):
        wrong_sample.append(i)

y_label =[1 for x in range(0,len(pos_sample_and_neig_pruned))]
y_label.extend([2 for i in range(0,len(neg_sample_and_neig_pruned))])

final_y_label = [y_label[i] for i in range(0,len(pos_neg_sample)) if i not in wrong_sample]
final_y_label = np.asarray(final_y_label)
shuffle_y_label = shuffle(final_y_label,random_state=0)

final_train_dataset = [train_dataset[i] for i in range(0,len(pos_neg_sample)) if i not in wrong_sample]
final_train_dataset = np.vstack(np.asarray(final_train_dataset))
shuffle_train_dataset = shuffle(final_train_dataset,random_state=0)

# skf = StratifiedKFold(n_splits=3)
# for train, test in skf.split(shuffle_train_dataset,shuffle_y_label):
#     for clf in clfs:
#         clfs[clf].fit(shuffle_train_dataset[train],shuffle_y_label[train])
#         conf_mat = confusion_matrix(clfs[clf].predict(shuffle_train_dataset[test]),shuffle_y_label[test])
#         print(clf)
#         print(accuracy_score(clfs[clf].predict(shuffle_train_dataset[test]),shuffle_y_label[test]))

for clf in clfs:
    print(clf)
    scores = cross_val_score(clfs[clf],shuffle_train_dataset,shuffle_y_label,cv=10)
    print(sum(scores)/10)

# After using using CD-HIT to remove redundant samples

In [22]:
cd_hit_7 = pd.read_csv("retained_CDHIT_0.7_PTM.csv")

In [23]:
cd_hit_7.head()

,pdbID,Chain
0,1ln4,A
1,1lop,A
2,1m63,A
3,1m63,B
4,1m63,D


#### 6 neighbours 1:1 = pos:neg

In [30]:
pos_sample_and_neig = pd.DataFrame()
for key, value in modification_site_dic.items():
    for chain, sites in value.items():
        if chain in list(cd_hit_7[cd_hit_7["pdbID"]==key]["Chain"]):
            for site in sites:
                neigh = n_nearest_neighbour_by_seq(key,chain,site,1)
                if len(neigh) > 0:
                    pos_sample_and_neig = pd.concat([pos_sample_and_neig,neigh])

In [105]:
neg_sample_and_neig = pd.DataFrame()
for i in range(len(pos_sample_and_neig)):
    result_step1 = One_residue_retrieval(pos_sample_and_neig.iloc[i,2],pos_sample_and_neig.iloc[i,0],pos_sample_and_neig.iloc[i,1])
    
    result_step2 = result_step1.groupby(["ID"]).first() #duplicate discard

    result_step3 = result_step2[~result_step2.ID_2.isin(modification_site_dic[pos_sample_and_neig.iloc[i,0]][pos_sample_and_neig.iloc[i,1]])] # activate discard
    #print(result_step3)
    result_step4 = result_step3[result_step3["Res_2"]=="LYS"]
    #print(result_step4)
    #print(result_step4.sample(1,random_state = 1))
    #neg_sample_and_neig = pd.concat([neg_sample_and_neig,result_step4.sample(5,replace = True)])
    #num_total_neg = len(result_step4)
    #neg_sample_and_neig = pd.concat([neg_sample_and_neig,result_step4.sample(num_total_neg//3,replace = False)])
    neg_sample_and_neig = pd.concat([neg_sample_and_neig,result_step4.sample(1,replace = True)])

In [106]:
pos_sample_and_neig_pruned = pos_sample_and_neig.iloc[:,[0,1,2,3]].copy(deep=True)
pos_sample_and_neig_pruned["type"] = "Pos"
pos_sample_and_neig_pruned.columns = ['pdbID', 'chain', 'ID', 'Res', 'type']

In [107]:
neg_sample_and_neig_pruned = neg_sample_and_neig.iloc[:,[0,1,4,5]].copy(deep=True)
neg_sample_and_neig_pruned["type"] = "Neg"
neg_sample_and_neig_pruned.columns = ['pdbID', 'chain', 'ID', 'Res', 'type']

In [108]:
pos_neg_sample = pd.concat([pos_sample_and_neig_pruned,neg_sample_and_neig_pruned])

In [109]:
neighbour_information_list = []
num_pos = 0
num_neg = 0
for i in range(0,len(pos_neg_sample)):
    neigh = n_nearest_neighbour_by_seq(pos_neg_sample.iloc[i,0],pos_neg_sample.iloc[i,1],pos_neg_sample.iloc[i,2],3)
    if len(neigh) > 0:
        neigh = neigh.iloc[:,5:8].values.flatten()
        neighbour_information_list.append(np.insert(neigh,0,pos_neg_sample.iloc[i,3]))
        if pos_neg_sample.iloc[i,4] == "Pos":
            num_pos = num_pos + 1
        else:
            num_neg = num_neg + 1

In [115]:
train_dataset = []
for each_res in neighbour_information_list:
    one_res_data = []
    for element in each_res[1:]:
#        if isinstance(element,str):
#            one_res_data.extend(onehot_encoded.transform([[amino_acid_property_by_res[element]-1]])[0].tolist())
#        else:
#            one_res_data.append(element)
        if isinstance(element,str):
            pass
        else:
            one_res_data.append(element)    
    
    train_dataset.append(one_res_data)

    
    
    
wrong_sample = []
for i in range(0,len(train_dataset)):
    if(len(train_dataset[i])<12):
        wrong_sample.append(i)

y_label =[1 for x in range(0,num_pos)]
y_label.extend([2 for i in range(0,num_neg)])

final_y_label = [y_label[i] for i in range(0,num_pos+num_neg) if i not in wrong_sample]
final_y_label = np.asarray(final_y_label)
shuffle_y_label = shuffle(final_y_label,random_state=0)

final_train_dataset = [train_dataset[i] for i in range(0,num_pos+num_neg) if i not in wrong_sample]
final_train_dataset = np.vstack(np.asarray(final_train_dataset))
shuffle_train_dataset = shuffle(final_train_dataset,random_state=0)

# skf = StratifiedKFold(n_splits=3)
# for train, test in skf.split(shuffle_train_dataset,shuffle_y_label):
#     for clf in clfs:
#         clfs[clf].fit(shuffle_train_dataset[train],shuffle_y_label[train])
#         conf_mat = confusion_matrix(clfs[clf].predict(shuffle_train_dataset[test]),shuffle_y_label[test])
#         print(clf)
#         print(accuracy_score(clfs[clf].predict(shuffle_train_dataset[test]),shuffle_y_label[test]))

for clf in clfs:
    print(clf)
    scores = cross_val_score(clfs[clf],shuffle_train_dataset,shuffle_y_label,scoring='accuracy',n_jobs=5,cv=10)
    print(sum(scores)/10)

decision_tree
0.850094575786
naive_gaussian
0.507383016318
naive_mul
0.516525268142
K_neighbor
0.666410107878
bagging_knn
0.619910273584
bagging_tree
0.765751607314
random_forest
0.849577264468
adaboost
0.575962177207
gradient_boost
0.577476624191


In [94]:
for clf in clfs:
    print(clf)
    scores = cross_val_score(clfs[clf],shuffle_train_dataset,shuffle_y_label,scoring='roc_auc',n_jobs=5,cv=10)
    print(sum(scores)/10)

decision_tree
0.920042551434
naive_gaussian
0.529884264647
naive_mul
0.526226046834
K_neighbor
0.766126629673
bagging_knn
0.774110381047
bagging_tree
0.929423483083
random_forest
0.983199646256
adaboost
0.575411760563
gradient_boost
0.576959264098


In [112]:
for clf in clfs:
    print(clf)
    scores = cross_val_score(clfs[clf],shuffle_train_dataset,shuffle_y_label,scoring='precision',n_jobs=5,cv=10)
    print(sum(scores)/10)

decision_tree
0.793420547439
naive_gaussian
0.529235254364
naive_mul
0.544258432181
K_neighbor
0.664764793844
bagging_knn
0.627773430561
bagging_tree
0.713938381891
random_forest
0.801229200578
adaboost
0.565750710641
gradient_boost
0.569717699745


In [96]:
for clf in clfs:
    print(clf)
    scores = cross_val_score(clfs[clf],shuffle_train_dataset,shuffle_y_label,scoring='recall',n_jobs=5,cv=10)
    print(sum(scores)/10)

decision_tree
0.917924206214
naive_gaussian
0.144920255572
naive_mul
0.257174559821
K_neighbor
0.419843437546
bagging_knn
0.0964273520948
bagging_tree
0.703762863971
random_forest
0.882319660537
adaboost
0.0128664097937
gradient_boost
0.0168219284983


In [120]:
len(final_y_label)

3936